In [ ]:
import pandas as pd
import numpy as np
import wrds

import capu as k
print(dir(k))

runned!
['PCA', 'SMOTE', 'SimpleImputer', 'StandardScaler', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'create_pca_table', 'np', 'pca_on_train_and_transform_full_data', 'pd', 'plt', 'preprocess_and_apply_smote1', 'remove_outliers_iqr', 'scale_columns1']


In [ ]:
db = wrds.Connection(wrds_username='lisadekievit')
print("Connection set up")

# loading in data North America 2016
df = db.raw_sql("""select conm, funda.datadate, naicsh, 
                    ap, lct, oancf, ppent, revt, xint, xsga, gp,
                    ni, ceq, at, act, ebit, ebitda, sale, re, ch, che, dlc, wcap,
                    lt, cogs, invt, rect, fyear, epspx, prcc_f, au, auop, currtr,
                    dltt, csho, dd1, dvt, gdwl,
                    aqc,caps,ci,dpact,dt,dv,emp,epsfi,esubc,pstk,txt,xopr,mkvalt
                   from comp.funda
                   where funda.datadate >= '01/01/2016'
                    AND funda.datadate < '01/01/2024'
                    AND auop IS NOT NULL
                    AND auop != '0'
                    AND auop != '5'
                    AND auop != '2'
                    AND auop != '3'
                   """, 
                   date_cols=['datadate'])
print("Data has been loaded in")

Loading library list...
Done
Connection set up
Data has been loaded in


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72784 entries, 0 to 72783
Data columns (total 51 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   conm      72784 non-null  object        
 1   datadate  72784 non-null  datetime64[ns]
 2   naicsh    62826 non-null  float64       
 3   ap        67509 non-null  float64       
 4   lct       54287 non-null  float64       
 5   oancf     63995 non-null  float64       
 6   ppent     70741 non-null  float64       
 7   revt      72458 non-null  float64       
 8   xint      64352 non-null  float64       
 9   xsga      53773 non-null  float64       
 10  gp        64122 non-null  float64       
 11  ni        64122 non-null  float64       
 12  ceq       72596 non-null  float64       
 13  at        72763 non-null  float64       
 14  act       54229 non-null  float64       
 15  ebit      64121 non-null  float64       
 16  ebitda    62274 non-null  float64       
 17  sale      64

In [ ]:
df.describe()

,datadate,naicsh,ap,lct,oancf,ppent,revt,xint,xsga,gp,...,dpact,dt,dv,emp,epsfi,esubc,pstk,txt,xopr,mkvalt
count,72784,62826.000000,6.750900e+04,54287.000000,63995.000000,70741.000000,72458.000000,64352.000000,53773.000000,64122.000000,...,57914.000000,5.765300e+04,63294.000000,60866.000000,62633.000000,57462.000000,72498.000000,72443.000000,64122.000000,5.850600e+04
mean,2020-05-03 03:33:32.090569472,412724.803378,4.608210e+03,1483.713611,639.335067,1951.805406,4030.190411,208.226528,678.774441,1361.060264,...,1818.598172,3.056035e+03,150.113192,10.704581,-2.158893,-5.169958,132.040615,98.831205,3138.868350,5.942702e+03
min,2016-01-31 00:00:00,42.000000,0.000000e+00,0.000000,-79910.000000,0.000000,-24954.684000,-3775.000000,-16.521000,-17188.000000,...,-3.870000,0.000000e+00,-0.861000,0.000000,-261400.000000,-31530.688000,-252.000000,-21515.000000,-23105.164000,0.000000e+00
25%,2017-12-31 00:00:00,325414.000000,1.564000e+00,5.819500,-3.034000,2.566000,12.359500,0.201000,8.880000,0.483000,...,0.878000,3.020000e-01,0.000000,0.082000,-0.420000,0.000000,0.000000,0.000000,18.480500,5.265163e+01
50%,2019-12-31 00:00:00,483111.000000,1.619500e+01,52.530000,10.202000,32.209000,178.299000,6.165500,45.512000,63.013000,...,23.692000,3.396900e+01,0.000000,0.589000,0.000000,0.000000,0.000000,0.571000,142.807500,3.310031e+02
75%,2021-12-31 00:00:00,522120.000000,1.901180e+02,405.525000,196.653500,372.377000,1362.214250,59.082750,248.596000,502.266500,...,384.821000,7.395000e+02,19.571250,4.250000,1.450000,0.000000,0.000000,20.050500,1057.877750,2.038946e+03
max,2023-12-31 00:00:00,999990.000000,2.462303e+06,227561.873000,231786.000000,292684.091000,608481.000000,110941.000000,232427.000000,300271.000000,...,398766.643000,4.211684e+06,37701.000000,2300.000000,99490.000000,21345.000000,139966.000000,49860.000000,575742.000000,2.662326e+06
std,NaN,175431.043061,5.829023e+04,7428.140050,3955.909661,10236.069081,18570.421022,2139.739142,3517.561383,6468.061089,...,10918.400778,5.436433e+04,770.111412,47.692821,1170.561936,230.566105,2595.239077,705.554430,15714.231571,4.171897e+04


In [ ]:
len(df['conm'])

72784

In [ ]:
# filter on USD
df = df[df['currtr'] == 1]
df = df.drop('currtr', axis= 1)

In [ ]:
len(df['conm'])

67614

In [ ]:
# big 4 X23 BINARY and X140 
big_4_codes = ["4", "5", "6", "7"]
df['X23'] = df['au'].apply(lambda x: 1 if x in big_4_codes else 0)

df['X140'] = df['naicsh'].fillna('00').astype(str).str[:2]

# X81, X86 tm X88
df['X81'] = df['dd1']
df['X86'] = df['dvt']
df['X87'] = df['gdwl']
df['X88'] = df['gp']

# X90 - 
df['X90'] = df['aqc']
df['X91'] = df['caps']
df['X92'] = df['ci']
df['X93'] = df['dpact']
df['X94'] = df['dt']
df['X95'] = df['dv']
df['X96'] = df['emp']
df['X97'] = df['epsfi']
df['X98'] = df['esubc']
df['X99'] = df['pstk']
df['X100'] = df['txt']
df['X101'] = df['xopr']
df['X102'] = df['mkvalt']

# 28.10
df['X106'] = df['fyear'] - 2010
df['X107'] = df['act']
df['X108'] = df['ap']
df['X109'] = df['at']
df['X110'] = df['ceq']
df['X111'] = df['ch']
df['X112'] = df['che']
df['X113'] = df['cogs']
df['X114'] = df['csho']
df['X115'] = df['dlc']
df['X116'] = df['dltt']
df['X117'] = df['ebit']

df['X119'] = df['epspx']
df['X120'] = df['invt']
df['X121'] = df['lct']
df['X122'] = df['lt']
df['X123'] = df['ni']
df['X124'] = df['oancf']
df['X125'] = df['ppent']
df['X126'] = df['re']
df['X127'] = df['rect']
df['X128'] = df['revt']
df['X129'] = df['sale']
df['X130'] = df['wcap']
df['X131'] = df['xint']
df['X132'] = df['xsga']
df['X133'] = df['prcc_f']

# delete columns
df = df.drop(["ni", 'ceq', 'at', 'act', 'ebit', 'ebitda', 'sale',
              're', 'ch', 'che', 'dlc', 'wcap', 'lt', 'cogs', 'invt', 'gp',
              'rect', "epspx", 'prcc_f', 'au', "ap", 'lct', 'oancf', 'ppent', 
              'revt', 'xint', 'xsga', 'dltt','csho','dd1',
              'aqc','caps','ci','dpact','dt','dv','emp','epsfi','esubc','pstk','txt','xopr','mkvalt',
              'dvt','gdwl'], axis= 1)
print("Columns have been made")

Columns have been made


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67614 entries, 0 to 72783
Data columns (total 51 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   conm      67614 non-null  object        
 1   datadate  67614 non-null  datetime64[ns]
 2   naicsh    58110 non-null  float64       
 3   fyear     67614 non-null  int64         
 4   auop      67614 non-null  object        
 5   X23       67614 non-null  int64         
 6   X140      67614 non-null  object        
 7   X81       63206 non-null  float64       
 8   X86       67069 non-null  float64       
 9   X87       67085 non-null  float64       
 10  X88       59315 non-null  float64       
 11  X90       57275 non-null  float64       
 12  X91       64680 non-null  float64       
 13  X92       63725 non-null  float64       
 14  X93       53560 non-null  float64       
 15  X94       53338 non-null  float64       
 16  X95       58672 non-null  float64       
 17  X96       56149 n

In [ ]:
print("The dataframe has {} rows and {} columns".format(df.shape[0],df.shape[1]))

The dataframe has 67614 rows and 51 columns


In [ ]:
# Calculate the correlation matrix for numeric columns
nm_cols = df.select_dtypes(include='float64').columns
nm_cols = nm_cols.drop("naicsh")
correlation_matrix = df[nm_cols].corr()
print("Correlation Matrix:")
print(correlation_matrix)

Correlation Matrix:
           X81       X86       X87       X88       X90       X91       X92  \
X81   1.000000  0.478714  0.289157  0.454028  0.108296  0.439520  0.355000   
X86   0.478714  1.000000  0.520334  0.721128  0.211969  0.484684  0.638753   
X87   0.289157  0.520334  1.000000  0.607109  0.377484  0.642493  0.491193   
X88   0.454028  0.721128  0.607109  1.000000  0.222233  0.586639  0.760690   
X90   0.108296  0.211969  0.377484  0.222233  1.000000  0.207076  0.124706   
X91   0.439520  0.484684  0.642493  0.586639  0.207076  1.000000  0.472136   
X92   0.355000  0.638753  0.491193  0.760690  0.124706  0.472136  1.000000   
X93   0.281795  0.604665  0.368043  0.590953  0.145139  0.383485  0.444997   
X94   0.551778  0.353396  0.089978  0.360941  0.028501  0.095203  0.167323   
X95   0.389016  0.956087  0.548995  0.698601  0.217277  0.500566  0.614770   
X96   0.250222  0.345688  0.424960  0.610327  0.156633  0.313694  0.344575   
X97  -0.000761  0.001158  0.001045  0.001383

In [ ]:
# Set the threshold for "high" correlation
threshold = 0.7
high_corr = correlation_matrix[(correlation_matrix.abs() > threshold) & (correlation_matrix != 1)]
high_corr = high_corr.dropna(how='all').dropna(axis=1, how='all')
# Convert the correlation matrix to a tidy DataFrame of variable pairs and their correlation
high_corr_pairs = high_corr.stack().reset_index()
high_corr_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Variable 1'] < high_corr_pairs['Variable 2']]
print("Unique High Correlation Pairs:")
print(high_corr_pairs)

Unique High Correlation Pairs:
    Variable 1 Variable 2  Correlation
2          X86        X88     0.721128
3          X86        X95     0.956087
6          X88        X92     0.760690
23         X94        X99     0.970239
40        X100       X117     0.726808
..         ...        ...          ...
174       X129       X132     0.805523
176       X131        X94     0.951744
177       X131        X99     0.898055
182       X132        X88     0.928306
183       X132        X96     0.754190

[95 rows x 3 columns]


In [ ]:
# Calculate the correlation between numeric columns and 'auop'
nm_cols = df.select_dtypes(include='float64').columns
nm_cols = nm_cols.drop("naicsh")
correlation_with_auop = df[nm_cols].corrwith(df['auop']).abs()
# Filter for high correlations with the target
threshold = 0.7
high_corr_with_auop = correlation_with_auop[correlation_with_auop > threshold]
# Convert to a DataFrame for readability
high_corr_pairs_with_auop = high_corr_with_auop.reset_index()
high_corr_pairs_with_auop.columns = ['Variable', 'Correlation with auop']
print("High Correlation with Target Variable 'auop':")
print(high_corr_pairs_with_auop)

High Correlation with Target Variable 'auop':
Empty DataFrame
Columns: [Variable, Correlation with auop]
Index: []


In [ ]:
correlation_with_auop

X81     0.003292
X86     0.021963
X87     0.020118
X88     0.025061
X90     0.010660
X91     0.024537
X92     0.028159
X93     0.036762
X94     0.015510
X95     0.038631
X96     0.002825
X97     0.000021
X98     0.003197
X99     0.029707
X100    0.015971
X101    0.032558
X102    0.017464
X107    0.051011
X108    0.000113
X109    0.010839
X110    0.030959
X111    0.005367
X112    0.001596
X113    0.033660
X114    0.004745
X115    0.000854
X116    0.027944
X117    0.021027
X119    0.000011
X120    0.013063
X121    0.049109
X122    0.014540
X123    0.031698
X124    0.027606
X125    0.036985
X126    0.021316
X127    0.015517
X128    0.026013
X129    0.033506
X130    0.025941
X131    0.015743
X132    0.025162
X133    0.022975
dtype: float64

In [ ]:
correlation_with_auop.sort_values()

X119    0.000011
X97     0.000021
X108    0.000113
X115    0.000854
X112    0.001596
X96     0.002825
X98     0.003197
X81     0.003292
X114    0.004745
X111    0.005367
X90     0.010660
X109    0.010839
X120    0.013063
X122    0.014540
X94     0.015510
X127    0.015517
X131    0.015743
X100    0.015971
X102    0.017464
X87     0.020118
X117    0.021027
X126    0.021316
X86     0.021963
X133    0.022975
X91     0.024537
X88     0.025061
X132    0.025162
X130    0.025941
X128    0.026013
X124    0.027606
X116    0.027944
X92     0.028159
X99     0.029707
X110    0.030959
X123    0.031698
X101    0.032558
X129    0.033506
X113    0.033660
X93     0.036762
X125    0.036985
X95     0.038631
X121    0.049109
X107    0.051011
dtype: float64

In [ ]:
from scipy.stats import chi2_contingency

# Function to calculate Cramér's V
def cramers_v(x, y):
    contingency_table = pd.crosstab(x, y)
    chi2 = chi2_contingency(contingency_table)[0]
    n = contingency_table.sum().sum()
    return np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

In [ ]:
# Calculate Cramér's V for X23 and auop
cramers_v_x23_auop = cramers_v(df['X23'], df['auop'])
print(f"Cramér's V between X23 and auop: {cramers_v_x23_auop}")

Cramér's V between X23 and auop: 0.15562239071081785


In [ ]:
# Calculate Cramér's V for X140 and auop
cramers_v_x140_auop = cramers_v(df['X140'], df['auop'])
print(f"Cramér's V between X140 and auop: {cramers_v_x140_auop}")

Cramér's V between X140 and auop: 0.23299425593395884


In [ ]:
print(df['auop'].value_counts())

auop
1    47243
4    20371
Name: count, dtype: int64
